In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import joblib

# Load model dan scaler
model = load_model("ModelPremiKesehatan.h5", compile=False)
scaler_X = joblib.load("scaler_Kesehatan_X.pkl")
scaler_y_reg = joblib.load("scaler_Kesehatan_Y.pkl")

# Daftar kolom fitur sesuai training (urutan harus sama!)
kolom_fitur = [
    'Usia',
    'Diabetes',
    'Masalah Tekanan Darah',
    'Riwayat Transplantasi',
    'Penyakit Kronis',
    'Tinggi Badan cm',
    'Berat Badan kg',
    'Alergi',
    'Riwayat Kanker Keluarga',
    'Jumlah Operasi Besar'
]

def prediksi_premi_risiko(data_dict):
    # Buat DataFrame dari dict input
    input_df = pd.DataFrame([data_dict])
    
    # Pastikan hanya ambil kolom fitur yang sesuai urutan training
    input_df = input_df[kolom_fitur]
    
    # Scaling fitur input
    X_scaled = scaler_X.transform(input_df)
    
    # Prediksi model (output list: [risk_pred, premium_pred])
    risk_pred, premium_pred = model.predict(X_scaled)
    
    # Ambil kelas risiko hasil klasifikasi (index probabilitas tertinggi)
    kelas_risiko = np.argmax(risk_pred, axis=1)[0]
    
    # Skala balik hasil regresi premi
    premi_pred = scaler_y_reg.inverse_transform(premium_pred)[0][0]
    
    # Bulatkan premi ke integer
    premi_pred = int(round(premi_pred))
    
    return kelas_risiko, premi_pred

In [3]:
# Contoh penggunaan inferensi
sample_input = {
    'Usia': 45,
    'Diabetes': 0,
    'Masalah Tekanan Darah': 0,
    'Riwayat Transplantasi': 0,
    'Penyakit Kronis': 0,
    'Tinggi Badan cm': 155,
    'Berat Badan kg': 57,
    'Alergi': 0,
    'Riwayat Kanker Keluarga': 0,
    'Jumlah Operasi Besar': 0
}

kelas, premi = prediksi_premi_risiko(sample_input)
print(f"Kelas Risiko Prediksi: {kelas}")
print(f"Premi Prediksi: Rp {premi:,}".replace(",", "."))

c:\Users\T480\Documents\CAPSTONE\fixenv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 822ms/step
Kelas Risiko Prediksi: 0
Premi Prediksi: Rp 285.103
